In [1]:
import os
import json
import boto3
import requests
import pandas as pd
import pyarrow as pa
import awswrangler as wr
import pyarrow.parquet as pq

from io import BytesIO
from datetime import datetime
from botocore.exceptions import NoCredentialsError

In [2]:
# Criar um cliente para o AWS Secrets Manager com base no perfil
client = boto3.session.Session(profile_name='CRISTIAN_AWS').client('secretsmanager', region_name='us-east-1')

# Recuperar o segredo
response = client.get_secret_value(SecretId='env/openweather')
secret_data = response['SecretString']

# Analisar o conteúdo do segredo (assumindo que é um formato JSON)
secrets_dict = json.loads(secret_data)

# Obtenha o valor associado à chave 'OPEN_WEATHER_SECRECT'
open_weather_token = secrets_dict.get('OPEN_WEATHER_SECRECT', '')

In [3]:
API_LINK = f"https://api.openweathermap.org/data/2.5/weather?q=novo hamburgo,br&APPID={open_weather_token}"

In [4]:
requisicao = requests.get(API_LINK)

In [5]:
wheater_json = requisicao.json()

In [6]:
wheater_json

{'coord': {'lon': -51.1306, 'lat': -29.6783},
 'weather': [{'id': 803,
   'main': 'Clouds',
   'description': 'broken clouds',
   'icon': '04d'}],
 'base': 'stations',
 'main': {'temp': 301.19,
  'feels_like': 303.01,
  'temp_min': 300.77,
  'temp_max': 301.97,
  'pressure': 1013,
  'humidity': 63,
  'sea_level': 1013,
  'grnd_level': 1009},
 'visibility': 10000,
 'wind': {'speed': 3.72, 'deg': 121, 'gust': 6.01},
 'clouds': {'all': 66},
 'dt': 1704394236,
 'sys': {'type': 2,
  'id': 2020383,
  'country': 'BR',
  'sunrise': 1704356966,
  'sunset': 1704407320},
 'timezone': -10800,
 'id': 3456068,
 'name': 'Novo Hamburgo',
 'cod': 200}

In [7]:
# Convertendo o dicionário Python para uma string JSON
weather_json_string = json.dumps(wheater_json)

In [8]:
weather_json_string

'{"coord": {"lon": -51.1306, "lat": -29.6783}, "weather": [{"id": 803, "main": "Clouds", "description": "broken clouds", "icon": "04d"}], "base": "stations", "main": {"temp": 301.19, "feels_like": 303.01, "temp_min": 300.77, "temp_max": 301.97, "pressure": 1013, "humidity": 63, "sea_level": 1013, "grnd_level": 1009}, "visibility": 10000, "wind": {"speed": 3.72, "deg": 121, "gust": 6.01}, "clouds": {"all": 66}, "dt": 1704394236, "sys": {"type": 2, "id": 2020383, "country": "BR", "sunrise": 1704356966, "sunset": 1704407320}, "timezone": -10800, "id": 3456068, "name": "Novo Hamburgo", "cod": 200}'

Save raw JSON at Data-lake

In [ ]:
test = {"coord": {"lon": -51.1306, "lat": -29.6783}, "weather": [{"id": 801, "main": "Clouds", "description": "few clouds", "icon": "02n"}], "base": "stations", "main": {"temp": 293.48, "feels_like": 293.79, "temp_min": 293.34, "temp_max": 294.33, "pressure": 1013, "humidity": 85, "sea_level": 1013, "grnd_level": 1009}, "visibility": 10000, "wind": {"speed": 1.51, "deg": 88, "gust": 4.18}, "clouds": {"all": 16}, "dt": 1704344172, "sys": {"type": 2, "id": 2020383, "country": "BR", "sunrise": 1704356966, "sunset": 1704407320}, "timezone": -10800, "id": 3456068, "name": "Novo Hamburgo", "cod": 200}

In [ ]:
test

In [11]:
s3

Session(region_name='us-east-1')

In [16]:
profile_name = 'CRISTIAN_AWS'

# Configuração do cliente S3
s3 = boto3.session.Session(profile_name=profile_name)
# Substitua 'seu-bucket' e 'seu-arquivo' pelos valores reais
bucket_name = 'data-integration-projects/openweather-data-lake/raw-data'
# Leitura dos dados JSON do S3 usando o awswrangler
df = wr.s3.read_json(f's3://{bucket_name}/*', lines=True, boto3_session = s3)


In [17]:
df

,coord,weather,base,main,visibility,wind,clouds,dt,sys,timezone,id,name,cod
0,"{'lon': -51.1306, 'lat': -29.6783}","[{'id': 801, 'main': 'Clouds', 'description': ...",stations,"{'temp': 293.48, 'feels_like': 293.79, 'temp_m...",10000,"{'speed': 1.51, 'deg': 88, 'gust': 4.18}",{'all': 16},1704344172,"{'type': 2, 'id': 2020383, 'country': 'BR', 's...",-10800,3456068,Novo Hamburgo,200
0,"{'lon': -51.1306, 'lat': -29.6783}","[{'id': 802, 'main': 'Clouds', 'description': ...",stations,"{'temp': 297.41, 'feels_like': 298.01, 'temp_m...",10000,"{'speed': 1.99, 'deg': 108, 'gust': 3.2}",{'all': 35},1704371630,"{'type': 2, 'id': 2020383, 'country': 'BR', 's...",-10800,3456068,Novo Hamburgo,200
0,"{'lon': -51.1306, 'lat': -29.6783}","[{'id': 802, 'main': 'Clouds', 'description': ...",stations,"{'temp': 298.46, 'feels_like': 299.06, 'temp_m...",10000,"{'speed': 1.99, 'deg': 108, 'gust': 3.2}",{'all': 35},1704373596,"{'type': 2, 'id': 2020383, 'country': 'BR', 's...",-10800,3456068,Novo Hamburgo,200
0,"{'lon': -51.1306, 'lat': -29.6783}","[{'id': 802, 'main': 'Clouds', 'description': ...",stations,"{'temp': 299.41, 'feels_like': 299.41, 'temp_m...",10000,"{'speed': 2.71, 'deg': 122, 'gust': 3.67}",{'all': 46},1704376514,"{'type': 2, 'id': 2020383, 'country': 'BR', 's...",-10800,3456068,Novo Hamburgo,200
0,"{'lon': -51.1306, 'lat': -29.6783}","[{'id': 802, 'main': 'Clouds', 'description': ...",stations,"{'temp': 300.13, 'feels_like': 302.05, 'temp_m...",10000,"{'speed': 2.71, 'deg': 122, 'gust': 3.67}",{'all': 46},1704377123,"{'type': 2, 'id': 2020383, 'country': 'BR', 's...",-10800,3456068,Novo Hamburgo,200
0,"{'lon': -51.1306, 'lat': -29.6783}","[{'id': 802, 'main': 'Clouds', 'description': ...",stations,"{'temp': 300.13, 'feels_like': 302.05, 'temp_m...",10000,"{'speed': 2.71, 'deg': 122, 'gust': 3.67}",{'all': 46},1704377123,"{'type': 2, 'id': 2020383, 'country': 'BR', 's...",-10800,3456068,Novo Hamburgo,200
0,"{'lon': -51.1306, 'lat': -29.6783}","[{'id': 802, 'main': 'Clouds', 'description': ...",stations,"{'temp': 299.78, 'feels_like': 299.78, 'temp_m...",10000,"{'speed': 2.71, 'deg': 122, 'gust': 3.67}",{'all': 46},1704377714,"{'type': 2, 'id': 2020383, 'country': 'BR', 's...",-10800,3456068,Novo Hamburgo,200
0,"{'lon': -51.1306, 'lat': -29.6783}","[{'id': 802, 'main': 'Clouds', 'description': ...",stations,"{'temp': 299.78, 'feels_like': 299.78, 'temp_m...",10000,"{'speed': 2.71, 'deg': 122, 'gust': 3.67}",{'all': 46},1704377714,"{'type': 2, 'id': 2020383, 'country': 'BR', 's...",-10800,3456068,Novo Hamburgo,200
0,"{'lon': -51.1306, 'lat': -29.6783}","[{'id': 803, 'main': 'Clouds', 'description': ...",stations,"{'temp': 300.07, 'feels_like': 301.95, 'temp_m...",10000,"{'speed': 3.12, 'deg': 125, 'gust': 3.89}",{'all': 51},1704380049,"{'type': 2, 'id': 2020383, 'country': 'BR', 's...",-10800,3456068,Novo Hamburgo,200
0,"{'lon': -51.1306, 'lat': -29.6783}","[{'id': 803, 'main': 'Clouds', 'description': ...",stations,"{'temp': 300.07, 'feels_like': 301.95, 'temp_m...",10000,"{'speed': 3.12, 'deg': 125, 'gust': 3.89}",{'all': 51},1704380049,"{'type': 2, 'id': 2020383, 'country': 'BR', 's...",-10800,3456068,Novo Hamburgo,200


In [9]:
# Defina suas credenciais explicitamente (não recomendado)
profile_name = 'CRISTIAN_AWS'
s3 = boto3.session.Session(profile_name=profile_name).client('s3')
bucket_name = 'data-integration-projects'
directory_path = 'openweather-data-lake/raw-data/'
dia_da_exucucao = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
file_name = f'weather_data_{dia_da_exucucao}.json'
s3.put_object(Bucket=bucket_name, Key=directory_path + file_name, Body=weather_json_string)

{'ResponseMetadata': {'RequestId': '6K2KXBGY4SY50GB4',
  'HostId': 'NFr/B2ygS9ZBUHO5EKFnlYREAW+LC9H4ufjIhrH3shwC7OzpKDYRrhVSd0O9jby1/ycTHHli2mk=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'NFr/B2ygS9ZBUHO5EKFnlYREAW+LC9H4ufjIhrH3shwC7OzpKDYRrhVSd0O9jby1/ycTHHli2mk=',
   'x-amz-request-id': '6K2KXBGY4SY50GB4',
   'date': 'Thu, 04 Jan 2024 18:51:26 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"19038177778a72ad0162024359b6c2bc"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"19038177778a72ad0162024359b6c2bc"',
 'ServerSideEncryption': 'AES256'}

Import data from data-lake

In [ ]:
import s3fs
profile_name = 'CRISTIAN_AWS'
s3 = boto3.session.Session(profile_name=profile_name).client('s3')
bucket_name = 'data-integration-projects'
directory_path = 'openweather-data-lake/raw-data/'
fs = s3fs.S3FileSystem()

# Obtenha a lista de objetos no diretório especificado
response = s3.list_objects_v2(Bucket=bucket_name, Prefix=directory_path)
s3_clientdata = None
s3_list = []
# Verifique se há objetos no diretório
if 'Contents' in response:
    for obj in response['Contents']:
        if obj["Size"] != 0:
            path = "s3://data-integration-projects/" + obj['Key']
            print(path)
            with fs.open(path, 'rb') as f:
                s3_clientdata = json.load(f)
        s3_list.append(s3_clientdata)
            
df = pd.json_normalize(s3_list)

In [ ]:
df

In [ ]:
response

In [ ]:
df

In [ ]:
df

In [ ]:
path = "s3://data-integration-projects/" + response["Contents"][1]['Key']

In [ ]:
df_aux = pd.read_json(path, orient= 'records')

In [ ]:
path

In [ ]:
response["Contents"][1]["Key"].split("/")[-1]

In [ ]:
response

In [ ]:
s3.download_file(bucket_name, obj['Key'], local_file_path)

In [ ]:
file_name

In [ ]:
objects

In [ ]:
s3.download_file(bucket_name, file_name, local_file_path)

In [ ]:
 s3.download_file(bucket_name, file_name, local_file_path)

    print(f'Arquivo {file_name} baixado para {local_file_path}')

In [ ]:
weather_df = pd.json_normalize(wheater_json)

In [ ]:
weather_df

In [ ]:
df_lon_lat = pd.DataFrame(wheater_json['coord'], index=[0])

In [ ]:
df_lon_lat

In [ ]:
df_weather = pd.DataFrame(wheater_json['weather'], index=[0])

In [ ]:
df_weather

In [ ]:
df_main = pd.DataFrame(wheater_json['main'], index=[0])

In [ ]:
df_main

In [ ]:
selected_columns = [
    "base",
    "visibility",
    "dt",
    "timezone",
    "id",
    "name",
    "cod",
    "coord.lon",
    "coord.lat",
    "main.temp",
    "main.feels_like",
    "main.temp_min",
    "main.temp_max",
    "main.pressure",
    "main.humidity",
    "main.sea_level",
    "main.grnd_level",
    "wind.speed",
    "wind.deg",
    "wind.gust",
    "clouds.all",
    "sys.type",
    "sys.id",
    "sys.country",
    "sys.sunrise",
    "sys.sunset",
]

In [ ]:
weather_df = pd.DataFrame(weather_df)

In [ ]:
weather_df

In [ ]:
df_clumns_selected = weather_df[selected_columns]

In [ ]:
df_clumns_selected

In [ ]:
super_df = pd.concat([df_weather, df_lon_lat, df_main, df_clumns_selected], axis=0, ignore_index=True)

In [ ]:
super_df

In [ ]:
super_df.to_parquet("C:/Users/Cristian/Desktop/area_de_trabalho/super_df.parquet")

In [ ]:
dia_da_exucucao

Save at S3

In [ ]:
# Defina suas credenciais explicitamente (não recomendado)
profile_name = 'CRISTIAN_AWS'
s3 = boto3.session.Session(profile_name=profile_name).client('s3')

In [ ]:
bucket_name = 'cristian-teste-buckt-from-python-code'

In [ ]:
local_file_path = "C:/Users/Cristian/Desktop/area_de_trabalho/super_df.parquet"

In [ ]:
dia_da_exucucao = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')

In [ ]:
dia_da_exucucao

In [ ]:
s3_file_name = f'df_teste_{dia_da_exucucao}.parquet'

In [ ]:
s3_file_name

In [ ]:
s3.upload_file(local_file_path, bucket_name, s3_file_name)